In [1]:
%matplotlib notebook
import time
import numpy as np
import scipy as sp
import ipywidgets as widgets
from inspect import signature
from ipywidgets import interact, interact_manual
from IPython.display import display
from lmntools import LMNRegressor
from plot import PlotterLLM
from auxiliaries import paraboloid
from systools import TrainingPlotter
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

# Local Model Playground

In [2]:
# Benutzerdefnierte Funktionen
functions=dict(
    arctan=lambda u: 3 * np.arctan(2*u) + 5,
    poly=lambda u: (4+u)*(u+1)*(u-1)*(u-3)/14,
    trigonometric=lambda u: np.sin(2*np.pi*u/4) + 0.5 * np.cos(2*np.pi*u),
    sin=lambda u: 2*np.sin(u),
    affensattel=lambda u1, u2: (u1**3 - 3 * u1 * u2**2 ) / 200,
    paraboloid=paraboloid(N=4))

In [4]:
items = [widgets.ToggleButton(value=True, description='result', disabled=False, tooltip='Description'),
         widgets.ToggleButton(value=True, description='error', disabled=False, tooltip='Description'),
         widgets.ToggleButton(value=False, description='validity_function', disabled=False, tooltip='Description'),
         widgets.ToggleButton(value=False, description='models', disabled=False, tooltip='Description'),
         widgets.ToggleButton(value=False, description='report', disabled=False, tooltip='Description'),
         widgets.ToggleButton(value=False, description='volume', disabled=False, tooltip='Description'),
         widgets.ToggleButton(value=False, description='distribution', disabled=False, tooltip='Description')]

fig1 = figure(plot_width=300, plot_height=300, title='training_loss', y_axis_type="log")
fig2 = figure(plot_width=300, plot_height=300, title='validation_loss', y_axis_type="log")
fig3 = figure(plot_width=300, plot_height=300, title='model_ranges', y_axis_type="log", y_range=[10e-10, 2])
training_plotter = TrainingPlotter(fig1, fig2, fig3, types=['line', 'line', 'line'])

def lolimot(model_complexity, p, x_range, func, k):
    # TRAINING
    training_plotter.reset()
    
    llm = LMNRegressor(network='lolimot', local_models="linear", model_complexity=model_complexity,
                       plotter=training_plotter, random_state=42)
    llm.network.smoothness = p
    
    N = len(signature(func).parameters)
    k = int(np.power(k, 1/N).round())
    k_pred = int(np.clip(k**(2/N), a_min=1, a_max=10000))
    
    if N == 1:
        X_train = np.reshape(np.linspace(x_range[0], x_range[1], k), (k, 1))
        y_train = func(X_train).ravel()
        _ = llm.fit(X_train, y_train)
        
        # PLOT
        X_test = np.reshape(np.linspace(x_range[0], x_range[1], k_pred), (k_pred, 1))
        y_test = func(X_test)
        
    elif N == 2:
        x1, x2 = np.meshgrid(np.linspace(-5, 5, k), np.linspace(-5, 5, k))
        y_train = np.reshape(func(x1, x2), (k*k, ))
        print(f"Effective numbers of samples {y_train.shape[0]}.")
        X_train = np.vstack((x1.flatten(), x2.flatten())).T
        
        _ = llm.fit(X_train, y_train)
    
        # PLOT
        x1, x2 = np.meshgrid(np.linspace(-5, 5, k_pred), np.linspace(-5, 5, k_pred))
        y_test = np.reshape(func(x1, x2), (k_pred*k_pred, ))
        X_test = np.array([x1, x2])
    else:
        x = np.meshgrid(*[np.linspace(x_range[0], x_range[1], k) for _ in range(N)])
        y_train = np.reshape(func(*x), (k**N, ))
        print(f"Effective numbers of samples {y_train.shape[0]}.")
        X_train = np.vstack((x_i.flatten() for x_i in x)).T
        
        _ = llm.fit(X_train, y_train)
        
        # PLOT
        X_test = np.meshgrid(*[np.linspace(x_range[0], x_range[1], k_pred) for _ in range(N)])
        y_test = np.reshape(func(*x), (k_pred**N, ))

    plotter = PlotterLLM(options=[item.description for item in items if item.value], sort=False)
    plotter.update(X_test, y_test, X_train, y_train, llm)

# Widgets
complexity_range = widgets.IntSlider(
    continuous_update=False, min=1, max=200,
    layout=widgets.Layout(width='90%'),
    description="complexity")
p_range = widgets.FloatSlider(
    continuous_update=False, min=0.05, max=1.5,
    value=0.33, step=0.01, layout=widgets.Layout(width='90%'))
x_range = widgets.IntRangeSlider(
    continuous_update=False, min=-10, max=10,
    value=[-5, 5], layout=widgets.Layout(width='90%'),
    description="range")
functions_dd = widgets.Dropdown(
    options=functions)
sample_num = widgets.IntText(value=100, description='number of samples:', disabled=False)


print("Push 'Run Interact' to start:")
plot_options = widgets.Box([widgets.Label(value="plot options:")] + items)
display(plot_options)
interact_manual(lolimot, model_complexity=complexity_range, p=p_range, x_range=x_range, func=functions_dd, k=sample_num);

Push 'Run Interact' to start:


Box(children=(Label(value='plot options:'), ToggleButton(value=True, description='result', tooltip='Description'), ToggleButton(value=True, description='error', tooltip='Description'), ToggleButton(value=False, description='validity_function', tooltip='Description'), ToggleButton(value=False, description='models', tooltip='Description'), ToggleButton(value=False, description='report', tooltip='Description'), ToggleButton(value=False, description='volume', tooltip='Description'), ToggleButton(value=False, description='distribution', tooltip='Description')))

interactive(children=(IntSlider(value=1, continuous_update=False, description='complexity', layout=Layout(width='90%'), max=200, min=1), FloatSlider(value=0.33, continuous_update=False, description='p', layout=Layout(width='90%'), max=1.5, min=0.05, step=0.01), IntRangeSlider(value=(-5, 5), continuous_update=False, description='range', layout=Layout(width='90%'), max=10, min=-10), Dropdown(description='func', options={'arctan': <function <lambda> at 0x7f54fe5a0400>, 'poly': <function <lambda> at 0x7f54fe5a0488>, 'trigonometric': <function <lambda> at 0x7f54fe5a0510>, 'sin': <function <lambda> at 0x7f54fe5a0598>, 'affensattel': <function <lambda> at 0x7f54fe5a0620>, 'paraboloid': <function paraboloid.<locals>.f at 0x7f54fe5a06a8>}, value=<function <lambda> at 0x7f54fe5a0400>), IntText(value=100, description='number of samples:'), Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widget-interact',))